# check dssp output
generated by dssp: https://github.com/PDB-REDO/dssp

raw result: ../dataset/ProteinGym/dssp/dssp_results.txt

In [2]:
import os
import pandas as pd
df = pd.read_csv("../dataset/ProteinGym/reference_files/DMS_substitutions.csv")
print(df.columns)
with open("../dataset/ProteinGym/dssp/dssp_results.txt", "r") as f:
    lines = f.readlines()
print(lines[0])

Index(['DMS_id', 'DMS_filename', 'UniProt_ID', 'taxon', 'source_organism',
       'target_seq', 'seq_len', 'includes_multiple_mutants',
       'DMS_total_number_mutants', 'DMS_number_single_mutants',
       'DMS_number_multiple_mutants', 'DMS_binarization_cutoff',
       'DMS_binarization_method', 'first_author', 'title', 'year', 'jo',
       'region_mutated', 'molecule_name', 'selection_assay', 'selection_type',
       'MSA_filename', 'MSA_start', 'MSA_end', 'MSA_len', 'MSA_bitscore',
       'MSA_theta', 'MSA_num_seqs', 'MSA_perc_cov', 'MSA_num_cov', 'MSA_N_eff',
       'MSA_Neff_L', 'MSA_Neff_L_category', 'MSA_num_significant',
       'MSA_num_significant_L', 'raw_DMS_filename', 'raw_DMS_phenotype_name',
       'raw_DMS_directionality', 'raw_DMS_mutant_column', 'weight_file_name',
       'pdb_file', 'ProteinGym_version', 'raw_mut_offset',
       'coarse_selection_type'],
      dtype='object')
Q2N0S5_9HIV1.pdb	('A', (' ', 1, ' '))	(1, 'M', '-', 1.0, 360.0, 152.6, 0, 0.0, 2, -0.2, 0, 0

Besides 7 proteins that skipped currently (check README.md for detailed), all other dssp result have the same length as the sequence.

In [3]:
dssp_dic_new = {}
for i in range(len(df)):
    dms_id = df.iloc[i]["DMS_id"]
    pdb_id = df.iloc[i]["UniProt_ID"]
    sub_lines = [line for line in lines if pdb_id in line]
    residue_list = []
    dssp_list = []
    for line in sub_lines:
        dssp_information = line.split("\t")[2]
        residue_list.append(dssp_information.split(",")[1][2])
        dssp_list.append(dssp_information.split(",")[2][2])
    dssp_dic_new[pdb_id] = dssp_list

In [15]:
import random
dms_id_ignored = ["A0A140D2T1_ZIKV_Sourisseau_2019", "BRCA2_HUMAN_Erwood_2022_HEK293T", "CAS9_STRP1_Spencer_2017_positive", "P53_HUMAN_Giacomelli_2018_Null_Etoposide", "P53_HUMAN_Giacomelli_2018_Null_Nutlin", "P53_HUMAN_Giacomelli_2018_WT_Nutlin",
"POLG_HCVJF_Qi_2014","POLG_CXB3N_Mattenberger_2021"]
uniprot_id_ignored = [df[df["DMS_id"] == x]["UniProt_ID"].values[0] for x in dms_id_ignored]
random.seed(0)
uniprot_id = df["UniProt_ID"].values
uniprot_id = [x for x in uniprot_id if x not in uniprot_id_ignored]
random_uniprot_id = random.sample(uniprot_id, 10)
for random_id in random_uniprot_id:
    dssp_assignment = dssp_dic_new[random_id]
    helix_residues = [i+1 for i, ltr in enumerate(dssp_assignment) if ltr == 'H']
    sheet_residues = [i+1 for i, ltr in enumerate(dssp_assignment) if ltr == 'E']
    helix_str = "+".join([f"{x}" for x in helix_residues])
    sheet_str = "+".join([f"{x}" for x in sheet_residues])
    print(random_id)
    print(helix_str)
    print(sheet_str)

NCAP_I34A1
22+23+24+25+26+27+28+29+30+31+32+33+34+35+36+37+38+39+40+41+42+43+44+45+46+51+52+53+54+57+58+59+60+61+62+63+64+65+66+67+68+69+70+73+74+75+76+77+78+79+80+81+113+114+115+116+117+118+119+120+121+122+123+130+131+132+133+134+135+136+137+138+139+140+141+142+143+144+145+146+147+151+152+153+154+155+156+157+177+178+179+180+181+182+183+186+187+188+189+190+191+192+193+194+195+196+197+198+199+200+201+202+209+210+211+212+213+214+215+216+217+218+219+220+221+222+223+224+225+226+227+228+229+233+234+235+236+237+238+239+240+241+242+243+244+250+251+252+253+254+255+256+257+258+259+260+261+262+278+279+280+281+282+283+284+285+286+291+292+293+294+302+303+304+305+306+307+308+322+323+324+325+326+327+328+329+330+331+332+333+341+342+343+344+345+346+347+348+422+423+424+425+426+427+439+440+441+442+443+444+445+446+447+448+449
91+92+93+94+95+96+97+98+99+100+103+104+105+106+107+108+109+110+111+112+148+149+313+314+315+316+375+376+377+378+385+386+416+417+456+457+464+465+494+495
TRPC_SACS2
7+8+9+10+11+12+13+1

First 6 chosen files has been checked by pymol, and saved colored residues to google drive (https://drive.google.com/drive/u/0/folders/1Q0u0YY-tZ2dS3babpZFUwifEyGEJsQuz)

Example pymol code:
```
load ODP2_GEOSE.pdb
select helix, (resi 8+9+10+11+12+13+14+15+16+35+36+37+38+39+40+41+42+43)
color red, helix
select sheet, (resi 5+6+32+33)
color blue, sheet
save ODP2_GEOSE.pse
```

In [19]:
# consider the distribution of the secondary structure elements in the dataset
import numpy as np
from tqdm import tqdm
total_residues = 0
total_helix = 0
total_sheet = 0
total_single_mutation = 0
total_helix_mutation = 0
total_sheet_mutation = 0
total_mutation = 0
print(len(df))
for i in tqdm(range(len(df))):
    pdb_id = df.iloc[i]["UniProt_ID"]
    dms_id = df.iloc[i]["DMS_id"]
    multiple_mutations = df.iloc[i]["includes_multiple_mutants"]
    helix_average_rank = []
    sheet_average_rank = []
    other_average_rank = []
    helix_average_dms = []
    sheet_average_dms = []
    other_average_dms = []
    if (dms_id not in dms_id_ignored) & ~multiple_mutations:
        dssp_str = dssp_dic_new[pdb_id]
        total_residues += df.iloc[i]["seq_len"]
        helix_residues = [i+1 for i, ltr in enumerate(dssp_str) if ltr == 'H']
        sheet_residues = [i+1 for i, ltr in enumerate(dssp_str) if ltr == 'E']
        total_helix += len(helix_residues)
        total_sheet += len(sheet_residues)
        df_dms = pd.read_csv(f"../dataset/ProteinGym/substitution/{dms_id}.csv")
        rank = df_dms["DMS_score"].rank(ascending=False)
        helix_rank_list = []
        sheet_rank_list = []
        other_rank_list = []
        helix_dms_list = []
        sheet_dms_list = []
        other_dms_list = []
        for j in range(len(df_dms)):
            total_mutation += 1
            mutant = df_dms.iloc[j]["mutant"]
            DMS_score = df_dms.iloc[j]["DMS_score"]
            if ":" not in mutant: # only consider single point mutations
                res_pos = int(mutant[1:-1])
                if res_pos in helix_residues:
                    total_helix_mutation += 1
                    helix_rank_list.append(rank[j])
                    helix_dms_list.append(DMS_score)
                if res_pos in sheet_residues:
                    total_sheet_mutation += 1
                    sheet_rank_list.append(rank[j])
                    sheet_dms_list.append(DMS_score)
                else:
                    other_rank_list.append(rank[j])
                    other_dms_list.append(DMS_score)
                total_single_mutation += 1
        if len(helix_rank_list) != 0:
            helix_average_rank.append(np.mean(helix_rank_list))
            helix_average_dms.append(np.mean(helix_dms_list))
        if len(sheet_rank_list) != 0:
            sheet_average_rank.append(np.mean(sheet_rank_list))
            sheet_average_dms.append(np.mean(sheet_dms_list))
        if len(other_rank_list) != 0:
            other_average_rank.append(np.mean(other_rank_list))
            other_average_dms.append(np.mean(other_dms_list))
print(total_residues,total_helix,total_sheet)
print(total_mutation,total_single_mutation, total_helix_mutation, total_sheet_mutation)
print(helix_average_rank, sheet_average_rank, other_average_rank)
print(np.nanmean(helix_average_rank), np.nanmean(sheet_average_rank), np.nanmean(other_average_rank))
print(np.nanmean(helix_average_dms), np.nanmean(sheet_average_dms), np.nanmean(other_average_dms))

217


  0%|                                                               | 0/217 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████| 217/217 [01:01<00:00,  3.51it/s]

62184 21329 9650
567983 567983 205194 100011
[] [] []
nan nan nan
nan nan nan



/tmp/ipykernel_142033/1667394487.py:67: RuntimeWarning: Mean of empty slice
  print(np.nanmean(helix_average_rank), np.nanmean(sheet_average_rank), np.nanmean(other_average_rank))
/tmp/ipykernel_142033/1667394487.py:68: RuntimeWarning: Mean of empty slice
  print(np.nanmean(helix_average_dms), np.nanmean(sheet_average_dms), np.nanmean(other_average_dms))


In [20]:
len(other_dms_list)

613

In [24]:
#split sequence into three subsequences with secondary structure elements, and calculate the spearman correlation between the prediction results and the ground truth
from scipy.stats import spearmanr
import numpy as np
ProtSSN_result = "/scratch/user/yxliu/ecen766/course_project/benchmark/ProtSSN/result/ProteinGym/"
PST_result = "/scratch/user/yxliu/ecen766/course_project/dataset/ProteinGym/result/PST_combined_results.csv"
saprot_result = "/scratch/user/yxliu/ecen766/course_project/dataset/ProteinGym/result/SaProt/"
df_pst = pd.read_csv(PST_result)
helix_spearman_list = []
helix_spearman_list_pst = []
helix_spearman_list_saprot = []
sheet_spearman_list = []
sheet_spearman_list_pst = []
sheet_spearman_list_saprot = []
other_spearman_list = []
other_spearman_list_pst = []
other_spearman_list_saprot = []
for i in range(len(df)):
    dms_id = df.iloc[i]['DMS_id']
    includes_multiple_mutants = df.iloc[i]['includes_multiple_mutants']
    if (dms_id not in dms_id_ignored) and ~includes_multiple_mutants:
        helix_gt = []
        helix_pred = []
        helix_pred_pst = []
        helix_pred_saprot = []
        sheet_gt = []
        sheet_pred = []
        sheet_pred_pst = []
        sheet_pred_saprot = []
        other_gt = []
        other_pred = []
        other_pred_pst = []
        other_pred_saprot = []
        df_protssn = pd.read_csv(ProtSSN_result + dms_id + ".csv")
        gt = df_protssn["score"].tolist()
        mutation_sites = df_protssn["mutant"].tolist()
        mutation_sites = [int(x[1:-1]) for x in mutation_sites]
        pred_protssn = df_protssn["ProtSSN_k20_h512"].tolist()
        pred_pst = df_pst[df_pst["protein_id"] == dms_id]["y_score"].tolist()
        df_saprot = pd.read_csv(saprot_result + dms_id + "_result.csv")
        pred_saprot = df_saprot["mutant_score"].tolist()
        secondary_structure = dssp_dic_new[df.iloc[i]["UniProt_ID"]]
        helix_residues = [i+1 for i, ltr in enumerate(secondary_structure) if ltr == 'H']
        sheet_residues = [i+1 for i, ltr in enumerate(secondary_structure) if ltr == 'E']
        other_residues = [i+1 for i, ltr in enumerate(secondary_structure) if ltr != 'H' and ltr != 'E']
        #print(len(helix_residues), len(sheet_residues), len(other_residues))
        for i in range(len(mutation_sites)):
            if mutation_sites[i] in helix_residues:
                helix_gt.append(gt[i])
                helix_pred.append(pred_protssn[i])
                helix_pred_pst.append(pred_pst[i])
                helix_pred_saprot.append(pred_saprot[i])
            if mutation_sites[i] in sheet_residues:
                sheet_gt.append(gt[i])
                sheet_pred.append(pred_protssn[i])
                sheet_pred_pst.append(pred_pst[i])
                sheet_pred_saprot.append(pred_saprot[i])
            if mutation_sites[i] in other_residues:
                other_gt.append(gt[i])
                other_pred.append(pred_protssn[i])
                other_pred_pst.append(pred_pst[i])
                other_pred_saprot.append(pred_saprot[i])
        if len(helix_gt) > 1:
            helix_spearman_list.append(spearmanr(helix_gt, helix_pred)[0])
            helix_spearman_list_pst.append(spearmanr(helix_gt, helix_pred_pst)[0])
            helix_spearman_list_saprot.append(spearmanr(helix_gt, helix_pred_saprot)[0])
        if len(sheet_gt) > 1:
            sheet_spearman_list.append(spearmanr(sheet_gt, sheet_pred)[0])
            sheet_spearman_list_pst.append(spearmanr(sheet_gt, sheet_pred_pst)[0])
            sheet_spearman_list_saprot.append(spearmanr(sheet_gt, sheet_pred_saprot)[0])
        if len(other_gt) > 1:
            other_spearman_list.append(spearmanr(other_gt, other_pred)[0])
            other_spearman_list_pst.append(spearmanr(other_gt, other_pred_pst)[0])
            other_spearman_list_saprot.append(spearmanr(other_gt, other_pred_saprot)[0])

print(np.mean(helix_spearman_list), np.mean(sheet_spearman_list), np.mean(other_spearman_list))
print(np.mean(helix_spearman_list_pst), np.mean(sheet_spearman_list_pst), np.mean(other_spearman_list_pst))
print(np.mean(helix_spearman_list_saprot), np.mean(sheet_spearman_list_saprot), np.mean(other_spearman_list_saprot))

0.4268960862262724 0.4639369891412551 0.39223921477609297
0.4205463021689388 0.44100768912117366 0.3838832200967671
0.4474098928870079 0.48610363955282837 0.4168413728104807


In [70]:
# backup: original codes for pyDssp
with open("../dataset/ProteinGym/dssp/dssp.output", "r") as f:
    lines = f.readlines()
print(len(lines))
print(lines[0])
dssp_id_list = [line.split()[1].split("/")[-1].split(".")[0] for line in lines]
dssp_dic = {}
for line in lines:
    line = line.split()
    pdb_id = line[1].split("/")[-1].split(".")[0]
    if pdb_id not in dssp_dic:
        dssp_dic[pdb_id] = []
    dssp_dic[pdb_id] = line[0]
print(dssp_id_list)
for i in range(len(df)):
    DMS_id = df.iloc[i]["DMS_id"]
    pdb_id = df.iloc[i]["UniProt_ID"]
    if pdb_id in dssp_dic.keys():
        sequence_length = df.iloc[i]["seq_len"]
        dssp_sequence = dssp_dic[pdb_id]
        if len(dssp_sequence) != sequence_length:
            print(pdb_id, len(dssp_sequence), sequence_length)
    else:
        print(pdb_id)

#randomly select 10 proteins and check if their dssp result the same as pymol plot
dms_id_ignored = ["A0A140D2T1_ZIKV_Sourisseau_2019", "BRCA2_HUMAN_Erwood_2022_HEK293T", "CAS9_STRP1_Spencer_2017_positive", "P53_HUMAN_Giacomelli_2018_Null_Etoposide", "P53_HUMAN_Giacomelli_2018_Null_Nutlin", "P53_HUMAN_Giacomelli_2018_WT_Nutlin",
"POLG_HCVJF_Qi_2014"]
uniprot_id_ignored = [df[df["DMS_id"] == x]["UniProt_ID"].values[0] for x in dms_id_ignored]
import random
random.seed(0)
uniprot_id = df["UniProt_ID"].values
uniprot_id = [x for x in uniprot_id if x not in uniprot_id_ignored]
random_uniprot_id = random.sample(uniprot_id, 10)
for random_id in random_uniprot_id:
    dssp_assignment = dssp_dic[random_id]
    helix_residues = [i+1 for i, ltr in enumerate(dssp_assignment) if ltr == 'H']
    sheet_residues = [i+1 for i, ltr in enumerate(dssp_assignment) if ltr == 'E']
    helix_str = "+".join([f"{x}" for x in helix_residues])
    sheet_str = "+".join([f"{x}" for x in sheet_residues])
    print(random_id)
    print(helix_str)
    print(sheet_str)

197
--HHH--EEEEEEE-----EEEEEEEE-EEEEEEE-----EEEEE-EEEE---EE-EEEE-EEEEE--EEEE----------HHHH---EEEEEEEEEE-HHH-----EEEEEEEEEEEEE-EEEEEEE--HHHEEEEEEEEE-----HHHH-----------EEEEEEE--E-EEEEE-HHH-EEEEEE-------HHHEEEEEE--EEEEEEHHHHH-----EEE---------E-HHHHEEE--EE--EE-EEE----HHHHHHH----EEEEEE--EEEE---EEEEEEE------------E----EEE----EE-----EEEEEEE------EE--EEEE-------EE-EE-----EE------EEEEEEEE---EEEEEEE--HHH-EEEEEEE---HHHHHHHHHHHHHHHHHHHHHHHHH------HHHHHHHHHHHHHHHHHHHHH----HHHHHHHHHHHHHHHHH---HHHHHHHHHHHHHHHHHHHH--- /scratch/user/yxliu/dataset/landscape/ProteinGym/AF2_structures//A0A140D2T1_ZIKV.pdb

['A0A140D2T1_ZIKV', 'A0A192B1T2_9HIV1', 'A0A1I9GEU1_NEIME', 'A0A220GHA5_9CAUD', 'A0A247D711_LISMN', 'A0A2Z5U3Z0_9INFA', 'A4D664_9INFA', 'A4GRB6_PSEAI', 'A4_HUMAN', 'AACC1_PSEAI', 'ACE2_HUMAN', 'ADRB2_HUMAN', 'AICDA_HUMAN', 'AMFR_HUMAN', 'AMIE_PSEAE', 'ANCSZ', 'ARGR_ECOLI', 'B1LPA6_ECOSM', 'B2L11_HUMAN', 'BAC7', 'BBC1_YEAST', 'BCHB_CHLTE', 'BLAT_ECOLX', 'BRCA1_HUMAN', 'BRCA2_HUMAN_1001-2085', 'BRCA2_HUM